In [2]:
from sqlalchemy import create_engine, text
import os

# Define the database file path
DB_FILE = "data/medical.db"

# Ensure the 'data' directory exists
os.makedirs('data', exist_ok=True)

# Create the database engine
engine = create_engine(f"sqlite:///{DB_FILE}")

def execute_sql_query(query: str):
    """
    Connects to the medical.db database and executes the given SQL query.
    Returns the fetched results.
    """
    try:
        with engine.connect() as connection:
            result = connection.execute(text(query))
            # If the query returns rows (e.g., SELECT statements)
            if result.returns_rows:
                # Fetch column names for better readability
                columns = result.keys()
                rows = result.fetchall()
                # Return results as a list of dictionaries for clarity
                return [dict(zip(columns, row)) for row in rows]
            else:
                # For non-SELECT queries (INSERT, UPDATE, DELETE), return success message
                return f"Query executed successfully. Rows affected: {result.rowcount}"
    except Exception as e:
        return f"SQL Error: {e}"
    finally:
        # Dispose the engine to ensure all connections are closed
        engine.dispose()

# query = """SELECT * FROM doctors WHERE LOWER(specialization) LIKE LOWER('%x-ray%') OR LOWER(field_of_interest) LIKE LOWER('%x-ray%') OR LOWER(specialization) LIKE LOWER('%radiology%') OR LOWER(field_of_interest) LIKE LOWER('%radiology%') OR LOWER(specialization) LIKE LOWER('%imaging%') OR LOWER(field_of_interest) LIKE LOWER('%imaging%')"""

query = "SELECT * FROM institutions"
# query = "SELECT T1.full_name FROM doctors AS T1 INNER JOIN institutions AS T2 ON T1.institution_name = T2.name WHERE T1.specialization = 'Dentist' AND T2.address LIKE '%Oak Avenue%'"
# query = "select * from institutions where address  LIKE '%Oak Avenue%'"
# query = "SELECT institution_name FROM doctors WHERE full_name LIKE '%Ana Petrović%'"
result = execute_sql_query(query)
for row in result:
    print(row)


{'name': 'Evergreen Medical Institute', 'type': 'Public', 'address': '123 Dr. Zoran Đinđića, Belgrade, Central Serbia 11000'}
{'name': 'Pinnacle Healthcare Group', 'type': 'Private', 'address': '45 Bulevar Oslobođenja, Novi Sad, Vojvodina 21000'}
{'name': 'Grand Central Hospital', 'type': 'Public', 'address': '78 Vojvode Stepe, Niš, Southern and Eastern Serbia 18000'}
{'name': 'Apex Medical Center', 'type': 'Private', 'address': '101 Kralja Milana, Kragujevac, Šumadija and Western Serbia 34000'}
{'name': 'Unity Wellness Clinic', 'type': 'Public', 'address': '20 Trg Slobode, Subotica, Vojvodina 24000'}
{'name': 'The Wellness Hub', 'type': 'Public', 'address': '30 Karađorđeva, Valjevo, Šumadija and Western Serbia 14000'}
{'name': 'Bridgepoint Health Systems', 'type': 'Private', 'address': '40 Knez Mihailova, Belgrade, Central Serbia 11000'}
{'name': 'Horizon Health Center', 'type': 'Public', 'address': '55 Partizanska, Bor, Eastern Serbia 19210'}
{'name': 'Compassionate Care Hospital', '

In [22]:
import os
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY") # type: ignore

CHROMA_PERSIST_DIRECTORY = "./chroma_db"
CHROMA_DEFAULT_COLLECTION_NAME = "langchain"

def check_chroma_db_contents():
    """
    Connects to the ChromaDB and prints a summary of its contents.
    """
    if not os.path.exists(CHROMA_PERSIST_DIRECTORY):
        print(f"Chroma DB directory '{CHROMA_PERSIST_DIRECTORY}' does not exist.")
        print("Please run your RAG init script first to create the database.")
        return

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    try:
        vectordb = Chroma(
            persist_directory=CHROMA_PERSIST_DIRECTORY,
            embedding_function=embeddings,
            collection_name=CHROMA_DEFAULT_COLLECTION_NAME
        )

        client = vectordb._client
        collection = client.get_collection(name=CHROMA_DEFAULT_COLLECTION_NAME)

        total_documents = collection.count()
        print(f"Total documents in Chroma DB collection '{CHROMA_DEFAULT_COLLECTION_NAME}': {total_documents}")

        if total_documents == 0:
            print("The collection is empty.")
            return

        print("\n--- Sample Documents (first 5) ---")
        retrieved_items = collection.get(limit=5, include=['documents', 'metadatas'])

        for i, doc_content in enumerate(retrieved_items['documents']):
            doc_metadata = retrieved_items['metadatas'][i]
            doc_id = retrieved_items['ids'][i]
            print(f"--- Document ID: {doc_id} ---")
            print(f"Title: {doc_metadata.get('title', 'N/A')}")
            print(f"Content (first 100 chars): {doc_content[:100]}...")
            print("-" * 30)

    except Exception as e:
        print(f"An error occurred while checking Chroma DB: {e}")


if __name__ == "__main__":
    check_chroma_db_contents()

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\1156798037.py:25: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Total documents in Chroma DB collection 'langchain': 56

--- Sample Documents (first 5) ---
--- Document ID: 09808f93-2b8f-44b3-8e51-49cae14d0519 ---
Title: Hypertension Guidelines
Content (first 100 chars): Hypertension, or high blood pressure, is a common condition. Treatment includes lifestyle changes an...
------------------------------
--- Document ID: 7dd7307f-1287-472e-ae2e-cb3edc067e55 ---
Title: Choosing the Right Specialist
Content (first 100 chars): Choosing a specialist depends on the medical issue. Cardiologists treat heart problems, while neurol...
------------------------------
--- Document ID: 3878eae1-3665-4591-8c9c-fec2bb20eda7 ---
Title: Private vs Public Medical Institutions
Content (first 100 chars): Private institutions offer shorter waiting times and personalized care, but services are usually pai...
------------------------------
--- Document ID: 5ffa41a3-2275-410c-9de7-4bbc6dc27fa9 ---
Title: Understanding MRI Scans
Content (first 100 chars): MRI scans provide 